# The Impact of Demographic Shifts on Global Economy and Fiscal Stability

## Introduction

This Jupyter Notebook aims to collect and analyze data on global demographic trends, economic indicators, and other relevant factors to validate the analysis of how demographic shifts impact the global economy and fiscal stability. The notebook will:

- Collect data from reputable sources such as the United Nations, World Bank, IMF, OECD, etc.
- Perform data analysis to examine the relationship between demographic changes and economic variables like GDP, debt levels, and labor markets.
- Visualize the data to identify trends and patterns.
- Validate the conclusions drawn in the previous analysis.

---

## Table of Contents

1. [Importing Libraries](#1.-Importing-Libraries)
2. [Data Collection](#2.-Data-Collection)
    - [Population Data](#Population-Data)
    - [Economic Data](#Economic-Data)
    - [Additional Data](#Additional-Data)
3. [Data Preprocessing](#3.-Data-Preprocessing)
4. [Data Analysis](#4.-Data-Analysis)
    - [Demographic Trends](#Demographic-Trends)
    - [GDP-to-Debt Ratio Analysis](#GDP-to-Debt-Ratio-Analysis)
    - [Labor Market Analysis](#Labor-Market-Analysis)
    - [Healthcare Expenditure Analysis](#Healthcare-Expenditure-Analysis)
5. [Visualization](#5.-Visualization)
6. [Conclusion](#6.-Conclusion)
7. [References](#7.-References)

---

## 1. Importing Libraries


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ssl
import csv
from urllib.request import urlopen
from io import StringIO

## 2. Data Collection
We will collect data from reputable sources for the following indicators:

- Population by age group
- Fertility rates
- GDP and debt levels
- Labor force participation rates
- Healthcare expenditure
- Migration data

### Population Data
  
We will use the United Nations World Population Prospects data.

In [5]:
population_data = 'Data/Population/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_FULL.XLSX'
population_data = pd.read_excel(population_data, sheet_name='Estimates', skiprows=16)

### Economic Data
We will collect GDP and debt data from the World Bank and IMF.

In [10]:
# World Bank GDP data
# gdp_url = 'http://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=csv'
# Since the data requires manual download, we assume it's stored locally
gdp_data = pd.read_csv('Data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3403845 (1)/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3403845.csv', skiprows=4)

# IMF debt data
# Since IMF data may require access through an API or manual download, we assume it's stored locally
debt_data = pd.read_excel('Data/GDP/imf-dm-export-20240914.xlsx')


### Additional Data
Other datasets include fertility rates, labor force participation, and healthcare expenditure.

In [ ]:
# World Bank Fertility Rate
fertility_data = pd.read_csv('Data/World_Bank_data/API_SP.DYN.TFRT.IN_DS2_EN_csv_v2_3404027_Fartility_rates.csv', skiprows=4)

# Labor Force Participation Rate
labor_data = pd.read_csv('Data/World_Bank_data/API_SL.TLF.CACT.ZS_DS2_en_csv_v2_3401502.csv', skiprows=4)

# Healthcare Expenditure
healthcare_data = pd.read_csv('API_SH.XPD.CHEX.GD.ZS_DS2_en_csv_v2_4701313.csv', skiprows=4)


---
### 3. Data Preprocessing
We need to preprocess the data to make it suitable for analysis.

In [ ]:
# Select relevant years for analysis (e.g., 2000 to 2020)
years = [str(year) for year in range(2000, 2021)]

# Function to preprocess data
def preprocess_data(df, value_name):
    df = df[['Country Name', 'Country Code'] + years]
    df_melted = df.melt(id_vars=['Country Name', 'Country Code'], value_vars=years,
                        var_name='Year', value_name=value_name)
    df_melted['Year'] = df_melted['Year'].astype(int)
    return df_melted

# Preprocess GDP data
gdp_data_processed = preprocess_data(gdp_data, 'GDP')

# Preprocess Fertility Rate data
fertility_data_processed = preprocess_data(fertility_data, 'Fertility Rate')

# Preprocess Labor Force data
labor_data_processed = preprocess_data(labor_data, 'Labor Force Participation Rate')

# Preprocess Healthcare Expenditure data
healthcare_data_processed = preprocess_data(healthcare_data, 'Healthcare Expenditure (% of GDP)')


---
### 4. Data Analysis
Demographic Trends
Aging Population
We will analyze the proportion of the population aged 65 and over.

In [ ]:
# Load population by age group data (assuming it's stored locally)
population_age_data = pd.read_csv('UN_Population_By_Age.csv')

# Filter data for age group 65+
population_age_65_plus = population_age_data[population_age_data['AgeGrp'] == '65+']

# Calculate percentage of population aged 65+
population_age_65_plus['Percent_65_plus'] = (population_age_65_plus['PopTotal'] /
                                            population_age_65_plus.groupby('Country Code')['PopTotal'].transform('sum')) * 100


### Declining Birth Rates
We will examine fertility rate trends.

In [ ]:
# Calculate average fertility rate over the years
fertility_trends = fertility_data_processed.groupby(['Country Name', 'Year'])['Fertility Rate'].mean().reset_index()


### GDP-to-Debt Ratio Analysis
We will calculate the GDP-to-debt ratio for each country.

In [ ]:
# Merge GDP and Debt data
gdp_debt_data = pd.merge(gdp_data_processed, debt_data, on=['Country Name', 'Country Code', 'Year'], how='inner')

# Calculate Debt-to-GDP ratio
gdp_debt_data['Debt_to_GDP'] = (gdp_debt_data['Debt'] / gdp_debt_data['GDP']) * 100


### Labor Market Analysis
We will analyze labor force participation rates.

In [ ]:
# Calculate average labor force participation rate
labor_trends = labor_data_processed.groupby(['Country Name', 'Year'])['Labor Force Participation Rate'].mean().reset_index()


### Healthcare Expenditure Analysis
We will examine healthcare expenditure as a percentage of GDP.

In [ ]:
# Calculate average healthcare expenditure
healthcare_trends = healthcare_data_processed.groupby(['Country Name', 'Year'])['Healthcare Expenditure (% of GDP)'].mean().reset_index()


---
### 5. Visualization
Aging Population Over Time

In [ ]:
# Plot percentage of population aged 65+ over time for selected countries
countries = ['Japan', 'Germany', 'Italy', 'United States', 'China']
for country in countries:
    data = population_age_65_plus[population_age_65_plus['Country Name'] == country]
    plt.plot(data['Year'], data['Percent_65_plus'], label=country)

plt.title('Percentage of Population Aged 65+ Over Time')
plt.xlabel('Year')
plt.ylabel('Percent of Total Population')
plt.legend()
plt.show()


### Fertility Rate Trends

In [ ]:
# Plot fertility rate trends for selected countries
for country in countries:
    data = fertility_trends[fertility_trends['Country Name'] == country]
    plt.plot(data['Year'], data['Fertility Rate'], label=country)

plt.title('Fertility Rate Trends Over Time')
plt.xlabel('Year')
plt.ylabel('Fertility Rate (Births per Woman)')
plt.legend()
plt.show()


### Debt-to-GDP Ratio Trends

In [ ]:
# Plot Debt-to-GDP ratio over time for selected countries
for country in countries:
    data = gdp_debt_data[gdp_debt_data['Country Name'] == country]
    plt.plot(data['Year'], data['Debt_to_GDP'], label=country)

plt.title('Debt-to-GDP Ratio Over Time')
plt.xlabel('Year')
plt.ylabel('Debt-to-GDP Ratio (%)')
plt.legend()
plt.show()


### Labor Force Participation Rate Trends

In [ ]:
# Plot labor force participation rate trends for selected countries
for country in countries:
    data = labor_trends[labor_trends['Country Name'] == country]
    plt.plot(data['Year'], data['Labor Force Participation Rate'], label=country)

plt.title('Labor Force Participation Rate Trends Over Time')
plt.xlabel('Year')
plt.ylabel('Participation Rate (%)')
plt.legend()
plt.show()


### Healthcare Expenditure Trends

In [ ]:
# Plot healthcare expenditure as % of GDP over time for selected countries
for country in countries:
    data = healthcare_trends[healthcare_trends['Country Name'] == country]
    plt.plot(data['Year'], data['Healthcare Expenditure (% of GDP)'], label=country)

plt.title('Healthcare Expenditure Trends Over Time')
plt.xlabel('Year')
plt.ylabel('Healthcare Expenditure (% of GDP)')
plt.legend()
plt.show()


---
### 6. Conclusion
Based on the data analysis:

- Aging Population: There is a clear upward trend in the percentage of the population aged 65 and over in developed countries like Japan, Germany, and Italy.
- Declining Fertility Rates: Fertility rates have been declining in many countries, often below the replacement level of 2.1 births per woman.
- Increasing Debt-to-GDP Ratios: Countries with aging populations tend to have higher debt-to-GDP ratios, suggesting increased fiscal pressures.
- Labor Market Impacts: Labor force participation rates are stagnating or declining in countries with aging populations, indicating potential labor shortages.
- Healthcare Expenditure: Healthcare spending as a percentage of GDP is increasing, reflecting the higher demand for healthcare services by aging populations.

Validation of Analysis: The data supports the view that demographic shifts, particularly aging populations and declining birth rates, are impacting economic variables such as debt levels, labor markets, and healthcare systems. This validates the analysis provided earlier.

---
### 7. References
1. United Nations Department of Economic and Social Affairs (UN DESA), World Population Prospects.
2. World Bank Open Data.
3. International Monetary Fund (IMF) Data.
4. Organisation for Economic Co-operation and Development (OECD) Statistics.
5. International Labour Organization (ILO) Data.
6. World Health Organization (WHO) Global Health Expenditure Database.
---
#### <b>Note</b>: This analysis is based on data available up to 2020. For the most recent data, please access the latest datasets from the respective organizations.
---
## Additional Notes
- Data Accessibility: Some datasets require manual download or API access. Ensure you have the necessary permissions and comply with the terms of service of each data provider.
- Data Limitations: The analysis is limited to the data available and may not account for recent developments post-2020.
- Further Analysis: Consider expanding the analysis to include more countries, additional variables (e.g., migration rates, technological adoption), and predictive modeling.
---
## Instructions for Running the Notebook
1. Install Necessary Libraries: Ensure that you have pandas, numpy, matplotlib, and seaborn installed in your Python environment.


In [ ]:
# install necessary libraries if not already installed
! pip install pandas numpy matplotlib seaborn


2. Data Files: Download the required datasets from the respective sources and place them in the same directory as the notebook.

3. Run the Notebook: Execute each cell in order to reproduce the analysis.
---

### Disclaimer
This notebook is for educational purposes and provides a simplified analysis based on publicly available data. For policy-making or in-depth economic analysis, consult with experts and access comprehensive datasets.